# Download and process GLHYMPS data into a set of shapefiles at native resolution

In [1]:
import os
import sys
import shutil
import zipfile
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling                                                                                        

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path            = cs.read_from_config(config_file,'data_path')
geospatial_temp_path = cs.read_from_config(config_file,'geospatial_temp_path')
geology_path         = cs.read_from_config(config_file,'geology_path')
geology_url          = cs.read_from_config(config_file,'geology_url')
download_area        = cs.read_from_config(config_file,'geospatial_area')

### Setup

In [4]:
# Folders
download_folder = Path(data_path) / geospatial_temp_path / geology_path / 'download'
raw_folder = Path(data_path) / geospatial_temp_path / geology_path / 'raw'

In [5]:
download_folder.mkdir(parents=True, exist_ok=True)
raw_folder.mkdir(parents=True, exist_ok=True)

In [6]:
# Convert subsetting area into a usable GDAL setting
# subset_area = [lon_min, lon_max, lat_min, lat_max]
# GDAL window = [ulx, uly, lrx, lry]; [upper left x, upper left y, lower right x, lower right y]
# Mapping:
#   ulx = lon_min = subset_area[0]
#   uly = lat_max = subset_area[3]
#   lrx = lon_max = subset_area[1]
#   lry = lat_min = subset_area[2]
subset_coor = download_area.split(',')
window = [subset_coor[0], subset_coor[3], subset_coor[1], subset_coor[2]]

### Downloading

Unfortunately, this is a manual download because you'll need to accept a data license agreement (CC-BY 4.0). Such is life, I guess. Go here: https://borealisdata.ca/file.xhtml?fileId=72026&version=1.0



### Unzip

In [19]:
file = cs.find_files_in_folder(download_folder, '.zip')[0] # function returns a list, but we know there's only 1 thing in there

In [20]:
with zipfile.ZipFile(download_folder/file, 'r') as zip_ref:
    zip_ref.extractall(download_folder)

In [21]:
os.remove(download_folder/file)

### Subset

In [6]:
import geopandas as gpd
from shapely.geometry import box

In [9]:
gdb_file = download_folder / 'GLHYMPS' / 'GLHYMPS.gdb'

In [33]:
raw_file = raw_folder / 'glhymps.shp'

In [37]:
cs.process_glhymps(gdb_file,raw_file,download_area)

C:\Users\wmk934\AppData\Local\Temp\1\ipykernel_9088\3225398715.py:15: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  subset.to_file(shp_file)


### Remove temporary files

In [38]:
# Retain the readme
shutil.move(download_folder / 'GLHYMPS' / 'readme.txt', raw_folder / 'readme.txt')

WindowsPath('C:/Globus endpoint/CAMELS_spat/geospatial_temp/glhymps/raw/readme.txt')

In [39]:
# Remove the download folder to save space
shutil.rmtree(download_folder)